# Deep Neural Network

### Image Mode

In [105]:
IMAGE_MODE = 1      #1 for colour, 0 for grayscale, unchanged for -1

### Datasets locations:

In [106]:
train_csv = "..\\Datasets\\Resized_data_50_50\\train.csv"
test_csv = "..\\Datasets\\Resized_data_50_50\\test.csv"

N_CLASSES = 2

## Imports

In [107]:
import tensorflow as tf

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import cv2

## Loading the dataset

In [108]:
train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)

In [109]:
train_df.head()

,Image_Path,Parasitized
0,..\Datasets\Resized_data_50_50\Parasitized\50x...,1.0
1,..\Datasets\Resized_data_50_50\Parasitized\50x...,1.0
2,..\Datasets\Resized_data_50_50\Uninfected\50x5...,0.0
3,..\Datasets\Resized_data_50_50\Parasitized\50x...,1.0
4,..\Datasets\Resized_data_50_50\Parasitized\50x...,1.0


In [110]:
train_df.dtypes

Image_Path      object
Parasitized    float64
dtype: object

## Data and Labels

In [111]:
train_x = train_df['Image_Path'].to_numpy()
train_y = train_df['Parasitized'].to_numpy()

test_x  = test_df['Image_Path'].to_numpy()
test_y  = test_df['Parasitized'].to_numpy()

#### Load images from paths

In [112]:
def load_images(path_arr):
    '''Reads and loads images into a numpy array
    Returns: a numpy array'''
    arr = []
    for path in path_arr:
        arr.append(cv2.imread(path, IMAGE_MODE))
    
    return np.array(arr)


In [113]:
train_x = load_images(train_x)
test_x = load_images(test_x)

#### Checking shape

In [114]:
print(train_x.shape, test_x.shape, sep = '\n')

(24802, 50, 50, 3)
(2756, 50, 50, 3)


## DNN

In [115]:
# batch sizes
TRAIN_BATCH_SIZE = 128
TEST_BATCH_SIZE  = 64

In [116]:
train_x = train_x/255.0
test_x = test_x/255.0

train_ds = tf.data.Dataset.from_tensor_slices((train_x, train_y))
test_ds  = tf.data.Dataset.from_tensor_slices((test_x, test_y))

train_ds = train_ds.batch(TRAIN_BATCH_SIZE)
test_ds  = test_ds.batch(TEST_BATCH_SIZE)

In [117]:
height, width = train_x.shape[1], train_x.shape[2]
dims = len(train_x.shape)-1

In [118]:
type(train_ds)

tensorflow.python.data.ops.dataset_ops.BatchDataset

In [130]:
input_shape = train_x.shape[1:]
input_shape

(50, 50, 3)

In [133]:
dnn_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), padding = 'same', activation = 'relu', input_shape = input_shape),
    tf.keras.layers.MaxPool2D((3,3), padding = 'same'),

    tf.keras.layers.Conv2D(32, (3,3), padding = 'same', activation = 'relu'),
    tf.keras.layers.MaxPool2D((3,3), padding = 'same'),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(512, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(N_CLASSES, activation = 'sigmoid')
])

dnn_model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0003),
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

dnn_model.fit(train_ds, batch_size = TRAIN_BATCH_SIZE, epochs = 20, validation_data = test_ds, validation_batch_size= TEST_BATCH_SIZE )

Epoch 1/20
194/194 [==============================] - 2s 7ms/step - loss: 0.5817 - accuracy: 0.6897 - val_loss: 0.5170 - val_accuracy: 0.7417
Epoch 2/20
194/194 [==============================] - 1s 6ms/step - loss: 0.4335 - accuracy: 0.8000 - val_loss: 0.4061 - val_accuracy: 0.8008
Epoch 3/20
194/194 [==============================] - 1s 6ms/step - loss: 0.3140 - accuracy: 0.8669 - val_loss: 0.2858 - val_accuracy: 0.8788
Epoch 4/20
194/194 [==============================] - 1s 6ms/step - loss: 0.2446 - accuracy: 0.9049 - val_loss: 0.2361 - val_accuracy: 0.8999
Epoch 5/20
194/194 [==============================] - 1s 6ms/step - loss: 0.2019 - accuracy: 0.9243 - val_loss: 0.2026 - val_accuracy: 0.9184
Epoch 6/20
194/194 [==============================] - 1s 6ms/step - loss: 0.1763 - accuracy: 0.9360 - val_loss: 0.1878 - val_accuracy: 0.9267
Epoch 7/20
194/194 [==============================] - 1s 6ms/step - loss: 0.1584 - accuracy: 0.9433 - val_loss: 0.1757 - val_accuracy: 0.9343
Epoch 